### tensorboard: 可視化神經網路的工具
直接使用 lesson 3 簡單手寫分類的程式，然後做一點修改，使得該程式透過tensorboard可視化tensorflow 計算結果。

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#載入數據集
mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

#每一個批次的大小
batch_size = 100 

#計算一共有多少批次
n_batch = mnist.train.num_examples // batch_size  # // 在python中表示取商


############### 修改!
#命名空間 (為了使用 tensorboard)
with tf.name_scope('input'):
    #定義兩個placeholder，目的在於 train時候透過 feed 傳入 x_data 與 y_data
    x = tf.placeholder(tf.float32, [None, 784]) # 28 * 28 = 784
    y = tf.placeholder(tf.float32, [None, 10]) #輸出層，有十個神經元，每個神經元有一個激活值，十個激活值排成一個 1*10的向量



#建立一個簡單的神經網路 (只有輸出層，總共10個神經元)
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([1, 10]))
prediction = tf.nn.softmax(tf.matmul(x, W) + b)

#代價函數 : loss = mean((y - prediction)^2)
loss = tf.reduce_mean(tf.square(y - prediction))

#Gradient desent method  (learning rate = 0.2)
gd = tf.train.GradientDescentOptimizer(0.2)

#最小化 代價函數 (operator)
train = gd.minimize(loss)

#初始化變數 operator
init = tf.global_variables_initializer()

#如果y標籤最大的值，與prediction標籤最大的值相等，則回傳true
#結果存在一個 boolean 的變數中
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax 回傳一維張量中最大的值，所在的位置

#求準確率
#轉換資料型態 boolean 轉成 float，接著在取平均值，得到準確率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 

#開始training
with tf.Session() as sess:
    sess.run(init)
    
    ############### 修改! (為了使用 tensorboard)
    writer = tf.summary.FileWriter('logs/', sess.graph) #第一個參數是路徑(在當前的路徑下建立資料夾 logs)，存入的東西是 graph (圖)
    
    
    for epoch in range(1): #總共疊代一次，只為了用 tensorboard 顯示訓練的結果，並不要求準確率
        #每一次 outer loop 不一次拿所有的數據集，來做 Gradient desent，這就是 stochastic gradient descent
        for batch in range(n_batch): #每一個 outer loop 疊代 n_batch 個批次
            #利用 train.next_batch 函數，讀取一個batch的 x, y 存給 batch_xs, batch_ys
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed_dict = {x: batch_xs, y: batch_ys}  #拿來feed 的 dictionary
            sess.run(train, feed_dict)
        #每做完一次 outer loop 計算一次準確率
        outer_loop_feed_dict = {x: mnist.test.images, y: mnist.test.labels} #testing data feed dictionary
        acc = sess.run(accuracy, outer_loop_feed_dict)
        print("Iter =" + str(epoch) + ", Testing Accuracy =" + str(acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter =0, Testing Accuracy =0.8307


* **到程式目錄下的 logs 資料夾，會看到所儲存的計算結果(graph) 其檔名類似 events.out.tfevents.1542000233.DESKTOP-U1OLPTF**
* **使用命令提示字元(cmd) ，移動到與程式相同的硬碟路徑下**
* **在cmd中輸入以下**


